In [10]:
import pandas as pd
from gurobipy import *
import pandas as pd
from collections import OrderedDict


In [11]:
#m = Model()
data = pd.read_excel(r'data.xlsx')
#data = pd.read_excel(r'data.xlsx',skiprows=1)
#data.drop('Center',axis = 1,inplace = True)
data.Center1 = 0.001*data.Center1 + 1
data.Center2 = 0.001*data.Center2 + 1
data.Center3 = 0.001*data.Center3 + 1
new_row = {'State':'Supply', 'Center1':1500000, 'Center2':1200000, 'Center3':1350000}
#append row to the dataframe
data = data.append(new_row, ignore_index=True)
data
#print(data.dtypes) 

,State,Center1,Center2,Center3,Demand
0,Alabama,3.302,1.228,2.251,47798.0
1,Alaska,3.933,6.221,6.780,7103.0
2,Arizona,1.530,3.016,3.904,63921.0
3,Arkansas,2.903,1.630,2.546,29160.0
4,California,1.000,3.529,4.372,372540.0
5,Colorado,1.979,2.602,3.393,50292.0
6,Connecticut,4.263,1.971,1.110,35741.0
7,Delaware,4.059,1.705,1.366,8980.0
8,District of Columbia,3.954,1.618,1.462,6018.0
9,Florida,3.644,1.292,2.221,188014.0


In [12]:
# Create a new model
m = Model("transport_problem")

# Create variables
x = {}
for i in range(51):
    for j in range(1,4):
        x[i,j] = m.addVar(lb = 0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name='x%s_%s' % (i,j))
        
# Integrate new variables
m.update()

#objective
obj = LinExpr()
obj = quicksum(x[i,j]*data.loc[i][j] for i in range(51) for j in range(1,4))

#change to Minimization environment     
m.setObjective(obj, GRB.MINIMIZE)

            

In [13]:
# Add supply constraints
for j in range(1,4):
    m.addConstr(quicksum(x[i,j] for i in range(51)) <= data.loc[51][j], 'supply_%s' % (i))
    

In [14]:
# Add demand constraints
for i in range(51):
    m.addConstr(quicksum(x[i,j] for j in range(1,4)) >= data.loc[i][4], 'demand_%s' % (j))


In [18]:
# Optimize the model. The default ModelSense is to is to minimize the objective, which is what we want.
m.optimize()

print ('\nOptimal flows :')
for i in range(51):
    for j in range(1,4):
        print ('state ',i+1, '->','center', j, ':', x[i,j].x)


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 54 rows, 153 columns and 306 nonzeros
Model fingerprint: 0xfc63769d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+03, 2e+06]
Presolve time: 0.01s
Presolved: 54 rows, 153 columns, 306 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.087481e+06   0.000000e+00      0s
      58    4.7759137e+06   0.000000e+00   0.000000e+00      0s

Solved in 58 iterations and 0.01 seconds
Optimal objective  4.775913746e+06

Optimal flows :
state  1 -> center 1 : 0.0
state  1 -> center 2 : 47798.0
state  1 -> center 3 : 0.0
state  2 -> center 1 : 7103.0
state  2 -> center 2 : 0.0
state  2 -> center 3 : 0.0
state  3 -> center 1 : 63921.0
state  3 -> center 2 : 0.0
state  3 -> center 3 : 0.0
state  4 ->

In [23]:
#Allowable Coeff Increase/Decrease 
for i in range(51):
    for j in range(1,4):
        print('state ',i+1, '->','center', j, ':',round(x[i,j].SAObjUp-data.loc[i][j],3), round(data.loc[i][j] - x[i,j].SAObjLow,3))


state  1 -> center 1 : inf 1.708
state  1 -> center 2 : 0.657 1.594
state  1 -> center 3 : inf 0.657
state  2 -> center 1 : 2.654 3.933
state  2 -> center 2 : inf 2.654
state  2 -> center 3 : inf 2.847
state  3 -> center 1 : 1.852 1.53
state  3 -> center 2 : inf 1.852
state  3 -> center 3 : inf 2.374
state  4 -> center 1 : inf 0.907
state  4 -> center 2 : 0.55 1.996
state  4 -> center 3 : inf 0.55
state  5 -> center 1 : 2.895 1.0
state  5 -> center 2 : inf 2.895
state  5 -> center 3 : inf 3.372
state  6 -> center 1 : 0.989 1.979
state  6 -> center 2 : inf 0.989
state  6 -> center 3 : inf 1.414
state  7 -> center 1 : inf 3.153
state  7 -> center 2 : inf 1.227
state  7 -> center 3 : 1.227 1.11
state  8 -> center 1 : inf 2.693
state  8 -> center 2 : inf 0.705
state  8 -> center 3 : 0.705 1.366
state  9 -> center 1 : inf 2.492
state  9 -> center 2 : inf 0.522
state  9 -> center 3 : 0.522 1.462
state  10 -> center 1 : inf 1.986
state  10 -> center 2 : 0.563 1.658
state  10 -> center 3 : inf

In [22]:
#dual values
shadow_price = m.getAttr('Pi', m.getConstrs())
for i in range(54):
    print(round(shadow_price[i],3))

0.0
-0.366
0.0
1.594
3.933
1.53
1.996
1.0
1.979
1.11
1.366
1.462
1.658
1.366
3.795
1.614
2.024
1.892
2.314
2.459
1.727
1.968
1.285
1.432
1.0
2.008
2.571
1.787
2.096
2.005
2.413
1.267
1.094
1.276
1.977
1.314
1.718
2.52
1.81
2.352
1.479
1.466
1.044
1.564
2.453
1.667
2.47
1.585
1.175
1.598
1.687
1.701
2.274
1.979
